In [ ]:
import collections
import os.path
import os
import networkx as nx
import openslide
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import collections
import tqdm
from skimage.segmentation import slic, mark_boundaries

In [ ]:
guangzhou_root = "/mnt/s3/lhm/gz_tiff/"
# esca_img = esca_root + 'images'
# esca_mask = esca_root + 'masks'
images = glob.glob(guangzhou_root + '*/*.tiff')
masks = glob.glob(guangzhou_root + '/*.npy')

In [ ]:
slide = openslide.open_slide(guangzhou_root + '/DP500-COAD_READ-GDPH_P01/part2/1115574-10-HE-DX1.tiff')
print(slide.level_downsamples,slide.level_dimensions)
img = slide.read_region((0,0),0,(21912, 38437))
plt.imshow(img)
plt.show()

In [13]:
import shutil
import tqdm
slides = glob.glob("/mnt/s3/lhm/gz_tiff/DP500-COAD_READ-GDPH_P01/*/*.tiff")
dst = "/mnt/s3/lhm/GDPH/TIFF/"
for s in tqdm.tqdm(slides):
    fileName = s.split('/')[-1].split('.')[-2]
    if fileName.endswith('class') or fileName.endswith('instance'):
        continue
    shutil.copy(s,dst+fileName+'.tiff')
    

100%|██████████| 2892/2892 [1:16:06<00:00,  1.58s/it]


In [15]:
slides = glob.glob("/mnt/s3/lhm/GDPH/TIFF/*.tiff")
dst = "/mnt/s3/lhm/GDPH/patch/"
mask_root = "/mnt/s3/lhm/GDPH/GDPH-tissue-segmentation/"
for s in slides:
    fileName = s.split('/')[-1].split('.')[-2]
    if not os.path.exists(dst+fileName):
       os.mkdir(dst+fileName) 
    slide = openslide.open_slide(s)
    mask = cv2.imread(mask_root+fileName+'.png')
    print(slide.level_dimensions)
    print(mask.shape)
    continue
    

((29880, 28753),)
(14376, 14940, 3)
((26892, 25063),)
(12531, 13446, 3)
((31956, 27244),)
(13621, 15978, 3)
((41638, 43917),)
(21958, 20819, 3)
((27995, 30189),)
(15094, 13997, 3)


KeyboardInterrupt: 

In [ ]:
# for i in images:
#     img = cv2.imread(i)
#     code = i.split('/')[-1]
#     mask = cv2.imread(esca_mask + '/' + code, cv2.IMREAD_GRAYSCALE)
#     height = img.shape[0]
#     width = img.shape[1]
#     hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     lower_white = np.array([0, 0, 220])  # 假设threshold是一个你选择的值
#     upper_white = np.array([180, 255, 255])
#     lower_black = np.array([0, 0, 0])
#     upper_black = np.array([180, 255, 50])
#     # 创建掩模来分离白色和黑色背景
#     mask1 = cv2.inRange(hsv_image, lower_white, upper_white)
#     mask2 = cv2.inRange(hsv_image, lower_black, upper_black)
#     # 使用 cv2.bitwise_or 来合并掩模
#     mask_all = 255 - cv2.bitwise_or(mask1, mask2)
#     #缩小图像
#     contours, _ = cv2.findContours(mask_all, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     max_area = 0
#     max_contour = None
#     xm = 0
#     ym = 0
#     # 遍历轮廓，找到面积最大的外接矩形
#     for contour in contours:
#         x, y, w, h = cv2.boundingRect(contour)
#         area = w * h
#         if area > max_area:
#             max_area = area
#             max_contour = contour
# 
#     if max_contour is not None:
#         xm, ym, w, h = cv2.boundingRect(max_contour)
#         hsv_image = hsv_image[ym:ym + h, xm:xm + w, :]
#         img_org = img.copy()
#         plt.imshow(img_org)
#         plt.show()
#         img = img[ym:ym + h, xm:xm + w, :]
#         mask = mask[ym:ym + h, xm:xm + w]
#         height, width = h, w
#         plt.imshow(img)
#         plt.show()
#         cv2.imwrite(f'/mnt/s3/lhm/ESCA_crop/images/{code}', img)
#         plt.imshow(mask)
#         plt.show()
#         cv2.imwrite(f'/mnt/s3/lhm/ESCA_crop/masks/{code}', mask)

In [ ]:
import concurrent.futures
from tqdm import tqdm
from multiprocessing import Pool, Pipe, freeze_support

#=============================================================#
# 接口                                                        #
#-------------------------------------------------------------#
#   multi_process_exec 多进程执行                             #
#   multi_thread_exec  多线程执行                             #
#-------------------------------------------------------------#
# 参数：                                                      #
#   f         (function): 批量执行的函数                      #
#   args_mat  (list)    : 批量执行的参数                      #
#   pool_size (int)     : 进程/线程池的大小                   #
#   desc      (str)     : 进度条的描述文字                    #
#-------------------------------------------------------------#
# 例子：                                                      #
# >>> def Pow(a,n):        ← 定义一个函数（可以有多个参数）   #
# ...     return a**n                                         #
# >>>                                                         #
# >>> args_mat=[[2,1],     ← 批量计算 Pow(2,1)                #
# ...           [2,2],                Pow(2,2)                #
# ...           [2,3],                Pow(2,3)                #
# ...           [2,4],                Pow(2,4)                #
# ...           [2,5],                Pow(2,5)                #
# ...           [2,6]]                Pow(2,6)                #
# >>>                                                         #
# >>> results=multi_thread_exec(Pow,args_mat,desc='计算中')   #
# 计算中: 100%|█████████████| 6/6 [00:00<00:00, 20610.83it/s] #
# >>>                                                         #
# >>> print(results)                                          #
# [2, 4, 8, 16, 32, 64]                                       #
#-------------------------------------------------------------#

ToBatch = lambda arr, size: [arr[i * size:(i + 1) * size] for i in range((size - 1 + len(arr)) // size)]


def batch_exec(f, args_batch, w):
    results = []
    for i, args in enumerate(args_batch):
        try:
            if isinstance(args, (list, tuple, dict)):
                ans = f(*args)
            else:
                ans = f(args)
            results.append(ans)
        except Exception as e:
            print(e)
            results.append(None)
        w.send(1)
    return results


def multi_process_exec(f, args_mat, pool_size=5, desc=None):
    if len(args_mat) == 0: return []
    batch_size = max(1, int(len(args_mat) / 4 / pool_size))
    results = []
    args_batches = ToBatch(args_mat, batch_size)
    with tqdm(total=len(args_mat), desc=desc) as pbar:
        with Pool(processes=pool_size) as pool:
            r, w = Pipe(duplex=False)
            pool_rets = []
            for i, args_batch in enumerate(args_batches):
                pool_rets.append(pool.apply_async(batch_exec, (f, args_batch, w)))
            cnt = 0
            while cnt < len(args_mat):
                try:
                    msg = r.recv()
                    pbar.update(1)
                    cnt += 1
                except EOFError:
                    print('EOFError')
                    break
            for ret in pool_rets:
                for r in ret.get():
                    results.append(r)
    return results


def multi_thread_exec(f, args_mat, pool_size=5, desc=None):
    if len(args_mat) == 0: return []
    results = [None for _ in range(len(args_mat))]
    with tqdm(total=len(args_mat), desc=desc) as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=pool_size) as executor:
            futures = {executor.submit(f, *args): i for i, args in enumerate(args_mat)}
            for future in concurrent.futures.as_completed(futures):
                i = futures[future]
                ret = future.result()
                results[i] = ret
                pbar.update(1)
    return results

In [ ]:
guangzhou_root = "/mnt/medical-data/zsxm/datasets/广州组织分割数据集"

images = glob.glob(guangzhou_root + '/*.jpg')
masks = glob.glob(guangzhou_root + '/*.npy')

In [ ]:
def convert_numpy_img_to_superpixel_graph(img, code, n=1024):

    height = img.shape[0]
    width = img.shape[1]
    # white_pixels = (img[:, :, 2] > 254) & (img[:, :, 1] > 254) & (img[:, :, 0] > 254)
    # white_count = np.sum(white_pixels)
    # white_ratio = (white_count * 100) / (width * height)
    # if white_ratio > 97.5:
    #     return
    # print(white_ratio)
    gt = np.load(guangzhou_root + '/' + code + '_gray.npy',allow_pickle=True)

    # img = cv2.resize(img, (width // scale, height // scale))
    mask = (img[:, :, 0] > 0) & (img[:, :, 1] > 0) & (img[:, :, 2] > 0)
    # gt = cv2.resize(gt, (width // scale, height // scale))
    # plt.imshow(img)
    # plt.show()    
    # plt.imshow(gt)
    # plt.colorbar()
    # plt.show()

    segments = slic(img, n_segments=n, compactness=8, sigma=0.9, start_label=0,
                    mask=mask)
    np.save(f"/mnt/s3/lhm/guangzhou/seg_cell/{code}.npy", segments)
    num_of_nodes = np.max(segments) + 1
    nodes = {node: {"rgb_list": [], "r": [], "g": [], "b": [], "y": []} for node in range(num_of_nodes)}
    # get rgb values and positions
    for y in range(segments.shape[0]):
        for x in range(segments.shape[1]):
            node = segments[y, x]
            if node < 0:
                continue
            rgb = img[y, x, :]
            nodes[node]["r"].append(rgb[2])
            nodes[node]["g"].append(rgb[1])
            nodes[node]["b"].append(rgb[0])
            nodes[node]["y"].append(gt[y][x])

    for node in nodes:
        r_bin = np.bincount(nodes[node]["r"])
        r_bin = np.pad(r_bin, (0, 256 - len(r_bin)), 'constant', constant_values=(0, 0))
        g_bin = np.bincount(nodes[node]["g"])
        g_bin = np.pad(g_bin, (0, 256 - len(g_bin)), 'constant', constant_values=(0, 0))
        b_bin = np.bincount(nodes[node]["b"])
        b_bin = np.pad(b_bin, (0, 256 - len(b_bin)), 'constant', constant_values=(0, 0))
        nodes[node]["rgb_list"] = np.stack([r_bin, g_bin, b_bin]).ravel()
    G = nx.Graph()
    # compute node positions
    segments_ids = np.unique(segments)
    segments_ids = np.delete(segments_ids, np.where(segments_ids == -1))
    pos = np.array([np.mean(np.nonzero(segments == i), axis=1) for i in segments_ids])
    pos = pos
    pos = pos.astype(int)
    #pos[0]为height_y pos[1]为width_x
    for node in nodes:
        feature = nodes[node]['rgb_list']
        feature = feature / np.sum(feature)
        mean = np.mean(feature)
        std = np.std(feature)
        feature = (feature - mean) / std
        counts = np.bincount(nodes[node]['y'])
        most_common_element = np.argmax(counts)
        G.add_node(node, features=feature, label=most_common_element)
    # add edges
    vs_right = np.vstack([segments[:, :-1].ravel(), segments[:, 1:].ravel()])
    vs_below = np.vstack([segments[:-1, :].ravel(), segments[1:, :].ravel()])
    bneighbors = np.unique(np.hstack([vs_right, vs_below]), axis=1)
    for i in range(bneighbors.shape[1]):
        if bneighbors[0, i] == -1 or bneighbors[1, i] == -1:
            continue
        if bneighbors[0, i] != bneighbors[1, i]:
            G.add_edge(bneighbors[0, i], bneighbors[1, i])
    # add self loops
    for node in nodes:
        G.add_edge(node, node)
    # get edge_index
    m = len(G.edges)
    edge_index = np.zeros([2 * m, 2]).astype(np.int64)
    for e, (s, t) in enumerate(G.edges):
        edge_index[e, 0] = s
        edge_index[e, 1] = t
        edge_index[m + e, 0] = t
        edge_index[m + e, 1] = s
    # get features
    num_of_features = 768
    x = np.zeros([n, num_of_features]).astype(np.float32)
    y = np.zeros(n).astype(np.float32)
    for node in G.nodes:
        if node >= n:
            continue
        x[node] = G.nodes[node]["features"]
        y[node] = G.nodes[node]["label"]
    return x, y, edge_index, pos


In [ ]:
args_mat = glob.glob(guangzhou_root + '/*.jpg')
for i in args_mat:
    img = cv2.imread(i)
    code = i.split('/')[-1].split('.jpg')[0]
    print(code)
    convert_numpy_img_to_superpixel_graph(img, code)

In [ ]:
from utils.utils_single import read_points_from_xml


def process_img(i):
    img = cv2.imread(i)
    code = i.split('/')[-1].split('.jpg')[0]
    if not os.path.exists(guangzhou_root + '/' + code + '_gray.npy'):
        return 
    x, y, edge_index, pos = convert_numpy_img_to_superpixel_graph(img, code)
    res = dict({})
    res['x'] = x
    res['edge'] = edge_index
    res['pos'] = pos
    res['code'] = code
    res['nodey'] = y
    np.save(f'/mnt/s3/lhm/guangzhou/graph_cell/{code}.npy', res, allow_pickle=True)


process_img(images[0])


In [ ]:
args_mat = glob.glob(guangzhou_root + '/*.jpg')
# print(args_mat)
multi_process_exec(process_img, args_mat, 32, desc='processing')

In [ ]:
from collections import Counter

a = np.load('/mnt/s3/lhm/ESCA_crop/graph_tissue/TCGA-BH-A0RX-DX1_xmin69099_ymin22327_MPP-0.2500.npy',
            allow_pickle=True).item()
img = cv2.imread('/mnt/s3/lhm/ESCA_crop/images/TCGA-BH-A0RX-DX1_xmin69099_ymin22327_MPP-0.2500.png')
mask = cv2.imread('/mnt/s3/lhm/ESCA_crop/masks/TCGA-BH-A0RX-DX1_xmin69099_ymin22327_MPP-0.2500.png',
                  cv2.IMREAD_GRAYSCALE)
plt.axis('off')
plt.imshow(img)
plt.show()
plt.axis('off')
plt.imshow(mask)
plt.colorbar()
plt.show()
fs = a['x']
nodey = a['nodey']
showed_y = []
for x, y in zip(fs, nodey):
    if y in showed_y:
        continue
    # 定义阈值
    threshold = 1.8
    # 根据阈值设置颜色
    print(y)
    showed_y.append(y)
    colors = ['red' if value > threshold else 'blue' for value in x]
    idx = [i for i in range(len(x))]
    plt.title(f'tissue node type {int(y)}')
    plt.bar(idx, x, color=colors)
    plt.show()

In [ ]:
from collections import Counter

gs = glob.glob("/mnt/s3/lhm/guangzhou/graph_cell/*.npy")

for g in gs:
    a = np.load(g, allow_pickle=True).item()
    code = g.split("/")[-1].split('.npy')[0]
    segments = np.load("/mnt/s3/lhm/guangzhou/seg_cell/" + code+'.npy', allow_pickle=True)
    nodey = a['nodey']
    rebuild = np.zeros_like(segments,dtype=np.int32)
    for index,y in enumerate(nodey):
        rebuild[segments==index] = int(y)
    plt.subplot(1,3,1)
    plt.imshow(rebuild)
    # plt.show()
    img = cv2.imread(guangzhou_root+"/"+code+'.jpg')
    gt = np.load(guangzhou_root + '/' + code + '_gray.npy',allow_pickle=True)
    plt.subplot(1,3,2)
    plt.imshow(img)
    # plt.show()
    plt.subplot(1,3,3)
    plt.imshow(gt)
    plt.show()

In [ ]:
gs = glob.glob("/mnt/s3/lhm/guangzhou/graph_cell/*.npy")
map = {}
original_array = [0.0, 2.0, 6.0, 7.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]
for idx,y in enumerate(original_array):
    map[y] = idx


for g in gs:
    a = np.load(g, allow_pickle=True).item()
    nodey = a['nodey']
    print(nodey)
    # newy = []
    # for index,y in enumerate(nodey):
    #     newy.append(map[y])
    # a['nodey'] = newy
    # np.save(g,a)
    